In [ ]:
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.2 MB 971 kB/s 
     |████████████████████████████████| 588.3 MB 6.3 kB/s 
     |████████████████████████████████| 439 kB 44.2 MB/s 
     |████████████████████████████████| 6.0 MB 41.8 MB/s 
     |████████████████████████████████| 1.7 MB 45.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1

In [ ]:
!pip install wurlitzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow_decision_forests as tfdf

import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/7 SEMESTRE/BLOQUE 2/RETO/randomForest/viajes_universidades_por_hora_100km_final.csv')
df

,timestamp,universidad,comuna_origen,tipo_de_viaje,tipo_de_universidad,tiene_residencia,count,tamaño_uni,n_carreras
0,0,Campus Andrés Bello Uchile,Conchalí,0,0,1,1,45000,53
1,0,Campus Andrés Bello Uchile,Huechuraba,0,0,1,1,45000,53
2,0,Campus Andrés Bello Uchile,Lo Espejo,0,0,1,1,45000,53
3,0,Campus Andrés Bello Uchile,Providencia,0,0,1,7,45000,53
4,0,Campus Andrés Bello Uchile,Quilicura,0,0,1,1,45000,53
...,...,...,...,...,...,...,...,...,...
1823,23,UDP Facultad de economía y empresas,Colina,0,1,0,1,26400,14
1824,23,UDP Facultad de economía y empresas,Huechuraba,0,1,0,4,26400,14
1825,23,UDP Facultad de economía y empresas,Huechuraba,1,1,0,9,26400,14
1826,23,UDP Facultad de economía y empresas,Las Condes,0,1,0,1,26400,14


In [ ]:
#df = df[['timestamp','universidad','count']]

In [ ]:
labels = 'count'

In [ ]:
def split_dataset(dataset, test_ratio=0.25):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [ ]:
train_ds_pd, test_ds_pd = split_dataset(df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

1369 examples in training, 459 examples for testing.


In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label='count',task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label = 'count',task=tfdf.keras.Task.REGRESSION)

In [ ]:
model_1 = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION,num_trees=200,max_depth=50)
model_1

Use /tmp/tmpumbst0wx as temporary training directory


In [ ]:
model_1.fit(x=train_ds)

Reading training dataset...
Training dataset read in 0:00:00.512011. Found 1369 examples.
Training model...
Model trained in 0:00:01.293809
Compiling model...
Model compiled.


In [ ]:
model_1.summary()

Model: "random_forest_model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (8):
	comuna_origen
	n_carreras
	tamaño_uni
	tiene_residencia
	timestamp
	tipo_de_universidad
	tipo_de_viaje
	universidad

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.             "__LABEL"  8.979076 ################
    2.    "tiene_residencia"  8.541437 ###############
    3. "tipo_de_universidad"  8.242968 ##############
    4.         "tamaño_uni"  7.550784 #############
    5.          "n_carreras"  6.802417 ###########
    6.       "tipo_de_viaje"  5.475837 ########
    7.         "universidad"  3.541034 #####
    8.           "timestamp"  2.340170 ##
    9.       "comuna_origen"  1.065464 

Variable Importance: NU

In [ ]:
model_1.compile(metrics=['mape'])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 0s 210ms/step - loss: 0.0000e+00 - mape: 75.1418

loss: 0.0000
mape: 75.1418


In [ ]:
tfdf.model_plotter.plot_model_in_colab(model_1, tree_idx=0)

In [ ]:
model_1.make_inspector().evaluation()

Evaluation(num_examples=1403, accuracy=None, loss=None, rmse=5.329923912057533, ndcg=None, aucs=None, auuc=None, qini=None)

In [ ]:
model_1.make_inspector().training_logs()

[TrainLog(num_trees=1, evaluation=Evaluation(num_examples=669, accuracy=None, loss=None, rmse=13.992154770630767, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=11, evaluation=Evaluation(num_examples=1796, accuracy=None, loss=None, rmse=14.119167492936784, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=21, evaluation=Evaluation(num_examples=1811, accuracy=None, loss=None, rmse=14.128277169787433, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=31, evaluation=Evaluation(num_examples=1811, accuracy=None, loss=None, rmse=14.165512212296564, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=41, evaluation=Evaluation(num_examples=1811, accuracy=None, loss=None, rmse=14.009244215420843, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=51, evaluation=Evaluation(num_examples=1811, accuracy=None, loss=None, rmse=14.0451126838114, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=61, evaluati